<b>ССЫЛКА НА ДАШБОРД</b><br>http://redash.public.karpov.courses/public/dashboards/UPzQvkqxtoVpDQawCEVl6qIKCYJxNeMSwopFFwvK?org_slug=default

### Динамика новых пользователей и новых курьеров, динамика всех пользователей и всех курьеров.

In [ ]:
with t1 as(
select time::date as date,
       count(distinct user_id) as new_users
from user_actions
where (user_id, time) in (select user_id,
                                 min(time)
                          from user_actions
                          group by user_id)
group by date
),

t2 as(
select time::date as date,
       count(distinct courier_id) as new_couriers
from courier_actions
where (courier_id, time) in (select courier_id,
                                 min(time)
                          from courier_actions
                          group by courier_id)
group by date
)

select date,
       new_users,
       new_couriers,
       sum(new_users) over(order by date)::int as total_users,
       sum(new_couriers) over(order by date)::int as total_couriers
from t1
join t2 using(date)
order by date

### Прирост числа новых пользователей и курьеров, прирост общего числа пользователей и курьеров.


In [ ]:
with t1 as(
select time::date as date,
       count(distinct user_id) as new_users
from user_actions
where (user_id, time) in (select user_id,
                                 min(time)
                          from user_actions
                          group by user_id)
group by date
),

t2 as(
select time::date as date,
       count(distinct courier_id) as new_couriers
from courier_actions
where (courier_id, time) in (select courier_id,
                                 min(time)
                          from courier_actions
                          group by courier_id)
group by date
),

t3 as(
select date,
       new_users,
       new_couriers,
       sum(new_users) over(order by date)::int as total_users,
       sum(new_couriers) over(order by date)::int as total_couriers
from t1
join t2 using(date)
order by date
)

select date,
       new_users,
       new_couriers,
       total_users,
       total_couriers,
       round(new_users::decimal / lag(new_users, 1) over() * 100 - 100, 2) as new_users_change,
       round(new_couriers::decimal / lag(new_couriers, 1) over() * 100 - 100, 2) as new_couriers_change,
       round(total_users::decimal / lag(total_users, 1) over() * 100 - 100, 2) as total_users_growth,
       round(total_couriers::decimal / lag(total_couriers, 1) over() * 100 - 100, 2) as total_couriers_growth
from t3


### Число платящих пользователей, число активных курьеров, доли платящих пользователей в общем числе пользователей на текущий день, доли активных курьеров в общем числе курьеров на текущий день.

In [ ]:
with pay_act as(
select date,
      paying_users,
      active_couriers
from(
    select time::date as date,
           count(distinct user_id) as paying_users
    from user_actions
    where order_id not in (select order_id from user_actions where action = 'cancel_order')
    group by date
) as t1
join (select time::date as date,
      count(distinct courier_id) as active_couriers
      from courier_actions
      where order_id in (select order_id from courier_actions where action = 'deliver_order')
      group by date) as t2 using(date)
),

new_u as(
SELECT time::date as date,
        count(distinct user_id) as new_users
FROM   user_actions
WHERE  (user_id, time) in (SELECT user_id,
                                   min(time)
                           FROM   user_actions
                           GROUP BY user_id)
group by date
),

new_c as(
SELECT time::date as date,
        count(distinct courier_id) as new_couriers
FROM   courier_actions
WHERE  (courier_id, time) in (SELECT courier_id,
                                   min(time)
                           FROM   courier_actions
                           GROUP BY courier_id)
group by date
)

select date,
       paying_users,
       active_couriers,
       round(paying_users::decimal / sum(new_users) OVER(ORDER BY date) * 100, 2) as paying_users_share,
       round(active_couriers::decimal / sum(new_couriers) over(order by date) * 100, 2) as active_couriers_share
from pay_act
join new_u using(date)
join new_c using(date)
order by date

### Динамика общего числа заказов, числа первых заказов и числа заказов новых пользователей, а так же доли первых заказов и доли заказов новых пользователей в общем числе заказов.

In [ ]:
with all_orders as(
select time::date as date,
       count(order_id) as orders
from user_actions
where order_id not in (select order_id from user_actions where action = 'cancel_order')
group by date
order by date
),

first_orders as (
select time::date as date,
       count(distinct user_id) as first_orders
from user_actions
where order_id not in (select order_id from user_actions where action = 'cancel_order')
and (user_id, time::date) in (select distinct user_id, min(time::date) from user_actions where order_id not in (select order_id from user_actions where action = 'cancel_order') group by user_id)
group by date
order by date
),

new_users_orders as (
select time::date as date,
       count(order_id) as new_users_orders
from user_actions
join (select user_id,
      min(time::date) as first_date
from user_actions
group by user_id) as first_orders using(user_id)
where time::date =  first_date
and order_id not in (select order_id from user_actions where action = 'cancel_order')
group by date
)

select date,
       orders,
       first_orders,
       new_users_orders,
       round(first_orders::decimal / orders * 100, 2) as first_orders_share,
       round(new_users_orders::decimal / orders * 100, 2) as new_users_orders_share
from all_orders
join first_orders using(date)
join new_users_orders using(date)
order by date



### Доли пользователей с одним и несколькими заказами.

In [ ]:
with all_paying_users as(
    select time::date as date,
            count(distinct user_id) as paying_users
    from user_actions
    where order_id not in(select order_id from user_actions where action = 'cancel_order')
    group by date
),

users_with_1_order as(
select date,
       count(distinct user_id) as user_1_order
from(
    select time::date as date,
           user_id
    from user_actions
    where order_id not in (select order_id from user_actions where action = 'cancel_order')
    group by date, user_id
    having count(order_id) = 1) as t1
group by date
)

select date,
       round(user_1_order::decimal / paying_users * 100, 2) as single_order_users_share,
       round(100 - (user_1_order::decimal / paying_users * 100), 2) as several_orders_users_share
from all_paying_users
join users_with_1_order using(date)
order by date


### Динамика показателя cancel rate и числа успешных/отменённых заказов.

In [ ]:
with delivered_orders as (
select EXTRACT(HOUR FROM creation_time)::int as hour,
       count(order_id) as successful_orders
from orders
where order_id in (select order_id from courier_actions where action = 'deliver_order')
group by hour
order by hour
),

canceled_orders as(
select EXTRACT(HOUR FROM creation_time)::int as hour,
       count(order_id) as canceled_orders
from orders
where order_id in (select order_id from user_actions where action = 'cancel_order')
group by hour
order by hour
)

select hour,
       successful_orders,
       canceled_orders,
       round(canceled_orders::decimal / (successful_orders + canceled_orders), 3) as cancel_rate
from delivered_orders
join canceled_orders using(hour)

### Динамика числа пользователей и заказов на одного курьера.

In [ ]:
with user_by_date as(
select time::date as date,
       count(distinct user_id) as user_count
from user_actions
where order_id not in (select order_id from user_actions where action = 'cancel_order')
group by date
),

courier_by_date as(
select time::date as date,
       count(distinct courier_id) as courier_count
from courier_actions
where order_id in (select order_id from courier_actions where action = 'deliver_order')
group by date
),

order_by_date as(
select creation_time::date as date,
       count(order_id) as order_count
from orders
where order_id not in (select order_id from user_actions where action = 'cancel_order')
group by date
)

select date,
       round(user_count::decimal / courier_count, 2) users_per_courier,
       round(order_count::decimal / courier_count, 2) orders_per_courier
from user_by_date
join courier_by_date using(date)
join order_by_date using(date)
order by date

### Динамика среднего времени доставки заказов.

In [ ]:
select time::date as date,
       avg(deliver_time)::int as minutes_to_deliver
from courier_actions
join (
select order_id,
       extract(EPOCH FROM(max(time)- min(time)))/60 as deliver_time
from courier_actions
where order_id not in (select order_id from user_actions where action = 'cancel_order')
group by order_id) as t1 using(order_id)
group by date
order by date